In [18]:
import cv2
import pandas as pd
import numpy as np

import torch.nn as nn
import torch.optim as optim

from torchvision import transforms

import timm

from pytorch_lightning import Trainer, LightningModule

from torch.utils.data import Dataset, DataLoader

In [19]:
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train_df['path'] = train_df.apply(lambda x : '../input/petfinder-pawpularity-score/train/' + x['Id'] + ".jpg", axis=1)

In [20]:
train_df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,path
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,../input/petfinder-pawpularity-score/train/000...
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,../input/petfinder-pawpularity-score/train/000...
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,../input/petfinder-pawpularity-score/train/001...
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,../input/petfinder-pawpularity-score/train/001...
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,../input/petfinder-pawpularity-score/train/001...


In [21]:
def read_image_as_rgb(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

class PetDataset(Dataset):

    def __init__(self, paths, labels, transform_fn=None) -> None:
        self.paths = paths
        self.labels = labels

        if len(self.paths) != len(self.labels):
            raise Exception("[paths] and [labels] is not same size.")

        self.transform_fn = transform_fn


    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        
        img = read_image_as_rgb(self.paths[index])
        
        if self.transform_fn:
            img = self.transform_fn(img)

        return img, self.labels[index]

In [22]:
class PetModel(LightningModule):

    def __init__(self):
        super(PetModel, self).__init__()
        self.model = timm.create_model('resnet18')
        self.model.fc = nn.Linear(in_features=512, out_features=1)
        self.loss_fn = nn.BCEWithLogitsLoss()

    def training_step(self, train_batch, batch_idx):
        X, y = train_batch
        pred = self.model(X)
        loss = self.loss_fn(pred, y)
        return loss

    def validation_step(self, val_batch, batch_idx):
        X, y = val_batch
        pred = self.model(X)
        loss = self.loss_fn(pred, y)
        return loss

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters())
        return optimizer


In [40]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

X_data = np.array(train_df.path)

y_data = np.array(train_df.Pawpularity)
y_data = y_data / 100.0
y_data = np.expand_dims(y_data, axis=1)

mask = np.random.rand(len(X_data)) < 0.8

X_train = X_data[mask]
y_train = y_data[mask]

X_val = X_data[~mask]
y_val = y_data[~mask] 

train_data_loader = DataLoader(PetDataset(X_train, y_train, transform_fn=data_transforms['train']), batch_size=32, num_workers=2)
val_data_loader = DataLoader(PetDataset(X_val, y_val, transform_fn=data_transforms['val']), batch_size=16, num_workers=2)

len(train_data_loader.dataset), len(val_data_loader.dataset)

(7959, 1953)

In [42]:
# DataLoader?

In [43]:
net = PetModel()
net

PetModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, mo

In [44]:
trainer = Trainer()
trainer.fit(net, train_data_loader, val_data_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name    | Type              | Params
----------------------------------------------
0 | model   | ResNet            | 11.2 M
1 | loss_fn | BCEWithLogitsLoss | 0     
----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.708    Total estimated model params size (MB)


Epoch 0:  11%|█▏        | 42/372 [01:10<09:15,  1.68s/it, loss=0.67, v_num=4] 

In [36]:
help(Trainer.fit)

Help on function fit in module pytorch_lightning.trainer.trainer:

fit(self, model: 'pl.LightningModule', train_dataloaders: Union[torch.utils.data.dataloader.DataLoader, Sequence[torch.utils.data.dataloader.DataLoader], Sequence[Sequence[torch.utils.data.dataloader.DataLoader]], Sequence[Dict[str, torch.utils.data.dataloader.DataLoader]], Dict[str, torch.utils.data.dataloader.DataLoader], Dict[str, Dict[str, torch.utils.data.dataloader.DataLoader]], Dict[str, Sequence[torch.utils.data.dataloader.DataLoader]], pytorch_lightning.core.datamodule.LightningDataModule, NoneType] = None, val_dataloaders: Union[torch.utils.data.dataloader.DataLoader, Sequence[torch.utils.data.dataloader.DataLoader], NoneType] = None, datamodule: Optional[pytorch_lightning.core.datamodule.LightningDataModule] = None, train_dataloader=None, ckpt_path: Optional[str] = None) -> None
    Runs the full optimization routine.
    
    Args:
        model: Model to fit.
    
        train_dataloaders: A collection of 